## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-09-11-51-47 +0000,bbc,Ukraine suffers heaviest attack as Trump criti...,https://www.bbc.com/news/articles/c2k19q4j07zo
1,2025-07-09-11-43-19 +0000,bbc,'Shoot in the leg' - Kenyan leader orders poli...,https://www.bbc.com/news/articles/c9dgv5e6447o
2,2025-07-09-11-35-19 +0000,nypost,At least 3 dead after flash flooding hits New ...,https://nypost.com/2025/07/09/us-news/deadly-f...
3,2025-07-09-11-33-08 +0000,bbc,US cuts visa validity for most Nigerian applic...,https://www.bbc.com/news/articles/cx23ywg9w82o
4,2025-07-09-11-17-37 +0000,bbc,Musk's Grok chatbot praises Hitler and insults...,https://www.bbc.com/news/articles/c4g8r34nxeno


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2292/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
4,trump,15
20,new,7
65,tariffs,6
21,mexico,5
49,military,5


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
80,2025-07-08-13-59-19 +0000,nyt,Trump’s New Trade Threats Set Off Global Scram...,https://www.nytimes.com/2025/07/08/business/ec...,38
16,2025-07-09-10-00-00 +0000,nypost,"Trump, Elon Musk had ‘very troublesome ending,...",https://nypost.com/2025/07/09/us-news/trump-el...,37
26,2025-07-09-09-01-38 +0000,nyt,Trump Didn’t Always Tout Tariffs. Now He Sees ...,https://www.nytimes.com/2025/07/09/us/politics...,33
12,2025-07-09-10-10-05 +0000,nypost,Here’s what Trump and Obama laughed and talked...,https://nypost.com/2025/07/09/us-news/trump-in...,31
41,2025-07-09-05-04-56 +0000,nypost,Trump insists ‘no amnesty’ for migrant farmwor...,https://nypost.com/2025/07/09/us-news/trump-in...,30


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
80,38,2025-07-08-13-59-19 +0000,nyt,Trump’s New Trade Threats Set Off Global Scram...,https://www.nytimes.com/2025/07/08/business/ec...
14,26,2025-07-09-10-00-00 +0000,nypost,Secretary of State Marco Rubio gets glowing en...,https://nypost.com/2025/07/09/us-news/secretar...
2,19,2025-07-09-11-35-19 +0000,nypost,At least 3 dead after flash flooding hits New ...,https://nypost.com/2025/07/09/us-news/deadly-f...
37,19,2025-07-09-05-36-08 +0000,nypost,Kennedy grandson Jack Schlossberg mocks grandm...,https://nypost.com/2025/07/09/us-news/kennedy-...
43,19,2025-07-09-03-09-46 +0000,nypost,10 people charged for ‘organized attack’ on IC...,https://nypost.com/2025/07/08/us-news/10-peopl...
45,18,2025-07-09-02-52-23 +0000,nypost,Rep. Ilhan Omar’s activist daughter unemployed...,https://nypost.com/2025/07/08/us-news/rep-ilha...
58,18,2025-07-08-21-31-55 +0000,bbc,King and Macron toast 'ever closer' UK-France ...,https://www.bbc.com/news/articles/cvg87y6d5j4o
34,17,2025-07-09-06-19-35 +0000,nypost,Camp Mystic’s disaster plan OK’d by state insp...,https://nypost.com/2025/07/09/us-news/texas-fl...
42,17,2025-07-09-04-00-08 +0000,nypost,Long-lost bow of WWII ship found nearly 83 yea...,https://nypost.com/2025/07/09/us-news/long-los...
25,16,2025-07-09-09-15-25 +0000,nypost,Georgia teen Sarah Grace Patrick accused of ki...,https://nypost.com/2025/07/09/us-news/georgia-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
